# Laboratorio 3 Evaluado
## Integrantes
* Daniel Bortot
* Hualong Chiang
* Alfredo Fung


# Situación:

Gustavo no quiere seguir trabajando por una miseria de sueldo y ha decidido embarcarse en una iniciativa relacionada a finanzas. Este negocio consiste en ofrecer prestamos para personas que quieren comenzar negocios profesionales. Solo hay un problema... Gustavo no sabe nada del tema.

Pero no pasa nada, en un mundo post IA puede dejarle el trabajo a un modelo de inteligencia artificial, entonces los ha contratado para que diseñen un modelo que sea capaz de predecir si es seguro prestarle dinero a una persona o no.

Para echarles un cable, Gustavo consiguio una base de datos sobre [prestamos](https://drive.google.com/file/d/1toUeiwOyEdmFgrtz84DxgY6y1i9-XHny/view?usp=sharing) a ciudadanos americanos para que lo ayuden.

Objetivo del laboratorio:

Diseñar un modelo MLP (MultiLayer Perceptron) que sea capaz de determinar si se debe aprobar o no un prestamo a una persona. Adicionalmente, la entrega final debe incluir un programa que el usuario pueda utilizar para ingresar los datos claves y en función a ese input devolver una recomendación.

Por ejemplo algo así:

"Por favor escriba (en numeros) x variable:"

Usuario escribe Y.

"En base a la información ustedes deberia... "




## 1. Instalamos Keras

In [36]:
%pip install -U scikit-learn
%pip install -U pandas
%pip install -q -U keras-tuner
%pip install -U tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.72.1-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cache

  You can safely remove it manually.
  You can safely remove it manually.


## 2. Cargamos los datos

In [32]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd
import re


dataframe = pd.read_csv("./data/loan_data.csv")
print(f"\n📌 Columnas del archivo propuesto:\n{', '.join(dataframe.columns)}\n")

# Extraemos los datos de X y de Y
X = dataframe[['Text', 'Income', 'Credit_Score',"Loan_Amount","DTI_Ratio","Employment_Status"]]
Y = dataframe['Approval']

# Filtrar solo los textos aprobados
approved_texts = " ".join(dataframe[dataframe["Approval"] == "Approved"]["Text"]).lower()

# Limpiar texto (eliminar signos de puntuación)
approved_texts = re.sub(r'[^\w\s]', '', approved_texts)

# Separar palabras y contar frecuencia
word_counts = Counter(approved_texts.split())

# Otro criterio
# Obtener las 5 palabras más comunes de los textos aprobados En caso que se quiera hacer con ese criterio
# Si se quiere hacer para los mas comunes

# keywords = [word for word, count in word_counts.most_common(5)]

keywords=["start", "invest"]

print("🔎 Palabras clave más frecuentes:", keywords)

# Función para marcar si un texto contiene palabras clave
def contains_keywords(text, keywords):
    text = text.lower()
    return 1 if any(word in text for word in keywords) else 0

# Funcion para clasificar a las personas de empleados y no empleados
def is_employed(person):
  if(person=="employed"):
    return 1
  else:
    return 0

# Aplicar la función al DataFrame
X = X.copy() # Se coloca la copia para evitar que se lance el error al filtrar los datos
X["Text"] = dataframe["Text"].apply(lambda x: contains_keywords(x, keywords))

X["Employment_Status"] = dataframe["Employment_Status"].apply(lambda x: is_employed(x))

print("\n✅ DataFrame actualizado con Keyword_Flag:\n")

# Ver los valores asignados a cada estatus de empleo
print("🚀 Mapeo de valores de Employment_Status:",X.columns )


# Aplicar LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(Y)

# Mostrar mapeo de valores
mapping_approval = dict(zip(le.classes_, le.transform(le.classes_)))
print("🚀 Mapeo de valores de Approval:", mapping_approval)

# Ver los valores transformados
print("\n🔍 Valores de Y codificados:", Y)
print("✅ Cantidad de elementos en Y:", len(Y))

# Ver los valores asignados a cada estado de aprobación
mapping_approval = dict(zip(le.classes_, le.transform(le.classes_)))
print("🚀 Mapeo de valores de Approval:", mapping_approval)

print("\n📊 Dataframe creado:\n")
dataframe.head()


📌 Columnas del archivo propuesto:
Text, Income, Credit_Score, Loan_Amount, DTI_Ratio, Employment_Status, Approval

🔎 Palabras clave más frecuentes: ['start', 'invest']

✅ DataFrame actualizado con Keyword_Flag:

🚀 Mapeo de valores de Employment_Status: Index(['Text', 'Income', 'Credit_Score', 'Loan_Amount', 'DTI_Ratio',
       'Employment_Status'],
      dtype='object')
🚀 Mapeo de valores de Approval: {'Approved': np.int64(0), 'Rejected': np.int64(1)}

🔍 Valores de Y codificados: [1 1 1 ... 1 1 0]
✅ Cantidad de elementos en Y: 24000
🚀 Mapeo de valores de Approval: {'Approved': np.int64(0), 'Rejected': np.int64(1)}

📊 Dataframe creado:



,Text,Income,Credit_Score,Loan_Amount,DTI_Ratio,Employment_Status,Approval
0,I need a loan to pay for an international vaca...,26556,581,8314,79.26,employed,Rejected
1,I want to make home improvements like installi...,197392,389,111604,22.14,employed,Rejected
2,"I need a loan for home renovation, including a...",44561,523,34118,45.44,employed,Rejected
3,I need funds to buy new furniture and applianc...,190363,729,118757,10.22,unemployed,Rejected
4,I need a loan to start a small business.,61853,732,19210,44.13,employed,Approved


In [33]:
# Impresion del dataframe de los valores de X
print("\n📊 Dataframe creado (X):\n")
X.head()


📊 Dataframe creado (X):



,Text,Income,Credit_Score,Loan_Amount,DTI_Ratio,Employment_Status
0,0,26556,581,8314,79.26,1
1,0,197392,389,111604,22.14,1
2,0,44561,523,34118,45.44,1
3,0,190363,729,118757,10.22,0
4,1,61853,732,19210,44.13,1


In [34]:
# Impresion del dataframe de los valores de Y

print("\n📊 Dataframe creado (Y):\n")
print(Y)


📊 Dataframe creado (Y):

[1 1 1 ... 1 1 0]


## 3. Despues de haber cargado los datos se procede a normalizar los datos que hay

In [37]:
import numpy as np #numpy
from sklearn.model_selection import train_test_split #separador de casos para entrenamiento y prueba
from tensorflow import keras #Wrapper de tensorflow
from tensorflow.keras.models import Sequential #Para hacer modelos secuenciales
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from keras_tuner.tuners import RandomSearch #Usando el metodo de tuning RandomSearch


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()

# Se normalizan los datos a entrenar
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Converte y to categorical (one-hot encoding)
y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)



## 4. Se procede a entrenar el modelo separando los datos de entrada y salida

In [39]:
import numpy as np #numpy
from sklearn.model_selection import train_test_split #separador de casos para entrenamiento y prueba
from tensorflow import keras #Wrapper de tensorflow
from tensorflow.keras.models import Sequential #Para hacer modelos secuenciales
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from keras_tuner.tuners import RandomSearch #Usando el metodo de tuning RandomSearch

def build_model(hp):
    # Se define un modelo Secuencial
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(2, activation='softmax')) # Output layer with softmax for multi-class
    model.compile(optimizer=keras.optimizers.Adam(
        hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='financing_classification')

tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

best_model = tuner.hypermodel.build(best_hps)
best_model.fit(X_train, y_train, epochs=10, validation_split=0.2)

loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Accuracy on test set: {accuracy}")


Trial 5 Complete [00h 00m 38s]
val_accuracy: 0.9947916865348816

Best val_accuracy So Far: 0.9961805542310079
Total elapsed time: 00h 02m 43s
Best hyperparameters: {'units': 256, 'learning_rate': 0.0044558929106897766}
Epoch 1/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9425 - loss: 0.1290 - val_accuracy: 0.9839 - val_loss: 0.0386
Epoch 2/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9848 - loss: 0.0364 - val_accuracy: 0.9872 - val_loss: 0.0291
Epoch 3/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9887 - loss: 0.0268 - val_accuracy: 0.9911 - val_loss: 0.0222
Epoch 4/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9928 - loss: 0.0205 - val_accuracy: 0.9940 - val_loss: 0.0166
Epoch 5/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9925 - loss: 0.0182 - val_accuracy: 0.9927 - val_loss: 0.0167
Epoch 6/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9897 - loss: 0.0227 - val_accuracy: 0.9945 - val_loss: 0.0152
Epoch 7/10
48

## 5. Validamos los valores generedados

In [40]:
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

print("\n📊 🔮 **Análisis del Modelo de Predicción de Préstamos** 🔮 📊")
print("-" * 50)

# Mostrar valores únicos de `y_train`
print("🔍 Exploración de los datos:")
print(f"✅ Valores únicos en el conjunto de entrenamiento (Y): {np.unique(y_train)}\n")

# Obtener predicciones
y_pred = np.argmax(best_model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)

# Mostrar comparación de predicción vs valor real en algunas muestras
print("📈 Comparación entre Predicción y Realidad del modelo de Aprobacion de prestamos:")
print("-" * 50)
N = 10  # Número de muestras a imprimir
for i in range(min(N, len(y_test))):
  print(f"🔹 Sample {str(i+1).zfill(2)} : 🏦 Predicted={y_pred[i]} | Real={y_true[i]}")
print("")

# Calcular métricas de rendimiento
mse = mean_squared_error(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

print("-" * 50)
print("📊  Resumen del Rendimiento del Modelo 📊")
print(f"🎯 Exactitud (`Accuracy`): {accuracy:.4f}")
print(f"📉 Error Cuadrático Medio (`MSE`): {mse:.4f}")
print("-" * 50)
print("📢 Gracias por analizar el modelo. Ajusta los parámetros para mejorar los resultados! 🚀")



📊 🔮 **Análisis del Modelo de Predicción de Préstamos** 🔮 📊
--------------------------------------------------
🔍 Exploración de los datos:
✅ Valores únicos en el conjunto de entrenamiento (Y): [0. 1.]

150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step
📈 Comparación entre Predicción y Realidad del modelo de Aprobacion de prestamos:
--------------------------------------------------
🔹 Sample 01 : 🏦 Predicted=1 | Real=1
🔹 Sample 02 : 🏦 Predicted=1 | Real=1
🔹 Sample 03 : 🏦 Predicted=1 | Real=1
🔹 Sample 04 : 🏦 Predicted=1 | Real=1
🔹 Sample 05 : 🏦 Predicted=1 | Real=1
🔹 Sample 06 : 🏦 Predicted=1 | Real=1
🔹 Sample 07 : 🏦 Predicted=1 | Real=1
🔹 Sample 08 : 🏦 Predicted=0 | Real=0
🔹 Sample 09 : 🏦 Predicted=1 | Real=1
🔹 Sample 10 : 🏦 Predicted=1 | Real=1

--------------------------------------------------
📊  Resumen del Rendimiento del Modelo 📊
🎯 Exactitud (`Accuracy`): 0.9948
📉 Error Cuadrático Medio (`MSE`): 0.0052
--------------------------------------------------
📢 Gracias por analizar el modelo. Aj

## Extra: Guardmanos el mejor modelo generado para uso posterior

Esto lo elaboramos con fin de guardar la informacion del modelo mejor entrenado para casos futuros asi como un respaldo y no tenerlo en memoria RAM, sino teniendo un elemento en almacenamiento persistente.

In [41]:
import joblib

# Guardar el modelo en un archivo
joblib.dump(best_model, "modelo_prestamos.pkl")

# Guardar el Scaler en un archivo para el programa de interfaz humano
joblib.dump(scaler, "scaler.pkl")


print("✅ Modelo guardado exitosamente en 'modelo_prestamos.pkl'")
print("✅ Scaler guardado exitosamente en 'scaler.pkl'")

✅ Modelo guardado exitosamente en 'modelo_prestamos.pkl'
✅ Scaler guardado exitosamente en 'scaler.pkl'


## 6. Programa para prueba del modelo

In [42]:
import joblib  # Para cargar el modelo previamente entrenado
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np


# Cargar el modelo de clasificación previamente entrenado
modelo = joblib.load("modelo_prestamos.pkl")

# Hacemos un ciclo while para validar c/u de los campos de tal forma que se entregen los datos al modelo siendo validos
def solicitar_datos():
    def obtener_texto():
        while True:
            texto = input("📊 Ingrese el motivo del préstamo que quiere solicitar: ").strip().lower()
            if texto:
                return texto
            print("⚠️ Error: El motivo del préstamo no puede estar vacío.")

    def obtener_flotante(mensaje):
        while True:
            try:
                valor = float(input(mensaje))
                if valor > 0:  # Verifica que el número sea positivo
                    return valor
                print("⚠️ Error: Ingrese un número positivo.")
            except ValueError:
                print("⚠️ Error: Ingrese un valor numérico válido.")

    def obtener_puntaje_credito():
        while True:
            try:
                puntaje = int(input("📊 Ingrese su puntaje de crédito: "))
                if puntaje > 0:  # Verifica que el número sea positivo
                    return puntaje
                print("⚠️ Error: El puntaje debe ser un número positivo.")
            except ValueError:
                print("⚠️ Error: Ingrese un número entero válido.")

    def obtener_empleo():
        opciones_validas = ["employed", "unemployed"]
        while True:
            empleo = input("👔 Ingrese su estado de empleo (employed, unemployed): ").strip().lower()
            if empleo in opciones_validas:
                return empleo
            print("⚠️ Error: Estado de empleo inválido. Debe ser 'employed' o 'unemployed'.")

    # Solicitar datos con validaciones
    texto = obtener_texto()
    ingreso = obtener_flotante("💰 Ingrese sus ingresos mensuales: ")
    puntaje_credito = obtener_puntaje_credito()
    monto_prestamo = obtener_flotante("🏦 Ingrese el monto del préstamo solicitado: ")
    dti_ratio = obtener_flotante("🔄 Ingrese su ratio DTI (deuda/ingresos): ")
    empleo = obtener_empleo()
    print("-" * 50)
    print("✅ Ingreso de datos exitosamente")


    return [texto, ingreso, puntaje_credito, monto_prestamo, dti_ratio, empleo]

def predecir_aprobacion(datos):
  if not datos:
    print("⚠️ No hay datos disponibles. Verifica la entrada.")

  print("\n" + "📊" * 5 + " INICIO DEL PROCESO DE PREDICCIÓN " + "📊" * 5)
  print("-" * 50)
  print("🔄 Preparando los datos para la predicción...\n")

  # Transformar los datos de entrada
  datos_transformados = transformar_datos(datos)
  print("📥 Datos transformados correctamente.")

  # Convertir a numpy array asegurando la forma adecuada
  print("🔍 Ajustando los datos para la predicción...")
  datos_array = np.array(datos_transformados, dtype=np.float32).reshape(1, -1)
  print("✅ Datos listos para la predicción.\n")

  # Realizar la predicción
  print("🤖 Ejecutando el modelo de predicción... 🕒")
  prediccion = modelo.predict(datos_array)
  print("✅ Predicción completada.\n")
  print("-" * 50)

  # Imprimir resultados con formato más claro
  probabilidad_aprobacion = prediccion[0][0] * 100  # Convertir a porcentaje
  probabilidad_rechazo = prediccion[0][1] * 100  # Convertir a porcentaje

  print("📊 Resultados del análisis:")
  print(f"❌ Probabilidad de rechazo: {probabilidad_rechazo:.2f}%")
  print(f"✅ Probabilidad de aprobación: {probabilidad_aprobacion:.2f}%\n")

  # Determinar el resultado final
  print("📌 Evaluando la decisión final del sistema...\n")
  resultado = "❌ Préstamo rechazado" if np.argmax(prediccion) == 1 else "✅ Préstamo aprobado"
  print(f"🏦 Decisión del sistema: {resultado}\n")

  print("-" * 50)
  print("📢 Gracias por utilizar nuestro sistema de predicción de préstamos. ¡Esperamos haber sido de ayuda!")

def transformar_datos(datos) -> list:
    import pandas as pd
    from sklearn.preprocessing import StandardScaler

    # Crear un DataFrame temporal
    df = pd.DataFrame([datos], columns=["Text", "Income", "Credit_Score", "Loan_Amount", "DTI_Ratio", "Employment_Status"])

    keywords=["start", "invest"]

    print("🔎 Palabras clave más frecuentes:", keywords)

    # Función para marcar si un texto contiene palabras clave
    def contains_keywords(text, keywords):
      text = text.lower()
      return 1 if any(word in text for word in keywords) else 0

    # Funcion para clasificar a las personas de empleados y no empleados
    def is_employed(person):
      if(person=="employed"):
        return 1
      else:
        return 0

    df = df.copy() # Se coloca la copia para evitar que se lance el error al filtrar los datos

    # Se filtran los datos del texto por palabras claves
    df["Text"] = df["Text"].apply(lambda x: contains_keywords(x, keywords))

    # Se filtran los datos del estatus de empleo por palabras claves
    df["Employment_Status"] = df["Employment_Status"].apply(lambda x: is_employed(x))

    # Normalizar los datos numéricos
    scaler = joblib.load("scaler.pkl")  # Cargar el escalador previamente entrenado

    print("🚀 Columnas originales del scaler:", scaler.feature_names_in_)

    # Escalar los valores
    df_transformado = scaler.transform(df)

    return df_transformado.tolist()


# 🚀 Programa de interacción
if __name__ == '__main__':

  print("\n🔮 Sistema de predicción de aprobación de préstamos")
  print("\n")
  print("📌 Descripción del sistema:")
  print("🔹 Basado en una red neuronal de nivel básico con una capa profunda.")
  print("🔹 Usa `RandomSearch` para la optimización de hiperparámetros.")
  print("🔹 Modelo secuencial con activación `ReLU` y salida `softmax`.")
  print("🔹 Función de pérdida: `categorical_crossentropy`.")
  print("🔹 Optimizado con `Adam`, ajustando la tasa de aprendizaje.")

  print("\n📊 Hiperparámetros:")
  print("🔹 Número de unidades en la capa oculta: 32 a 512.")
  print("🔹 Tasa de aprendizaje: 0.0001 a 0.01.")
  print("🔹 Evaluación basada en `val_accuracy`.")
  print("🔹 Máximo de 5 pruebas y 3 ejecuciones por prueba.")

  print("\n👥 Autores del sistema:")
  print("🔹 Daniel Bortot")
  print("🔹 Hualong Chiang")
  print("🔹 Alfredo Fung")
  print(50*"-")
  print("📝 Por favor, ingrese los siguientes datos:")
  datos_usuario = solicitar_datos()
  predecir_aprobacion(datos_usuario)



🔮 Sistema de predicción de aprobación de préstamos


📌 Descripción del sistema:
🔹 Basado en una red neuronal de nivel básico con una capa profunda.
🔹 Usa `RandomSearch` para la optimización de hiperparámetros.
🔹 Modelo secuencial con activación `ReLU` y salida `softmax`.
🔹 Función de pérdida: `categorical_crossentropy`.
🔹 Optimizado con `Adam`, ajustando la tasa de aprendizaje.

📊 Hiperparámetros:
🔹 Número de unidades en la capa oculta: 32 a 512.
🔹 Tasa de aprendizaje: 0.0001 a 0.01.
🔹 Evaluación basada en `val_accuracy`.
🔹 Máximo de 5 pruebas y 3 ejecuciones por prueba.

👥 Autores del sistema:
🔹 Daniel Bortot
🔹 Hualong Chiang
🔹 Alfredo Fung
--------------------------------------------------
📝 Por favor, ingrese los siguientes datos:
⚠️ Error: Estado de empleo inválido. Debe ser 'employed' o 'unemployed'.
--------------------------------------------------
✅ Ingreso de datos exitosamente

📊📊📊📊📊 INICIO DEL PROCESO DE PREDICCIÓN 📊📊📊📊📊
--------------------------------------------------
🔄 

## 7. Conclusiones

### Resume los principales hallazgos.

Consideramos que el modelo es efectivo porque cumple con la propuesta del laboratorio establecido. Ademas de hacer una interfaz humano computador decente para que sea mas amigable para el usuario.

Consideramos que una de las mejoras para hacer es el uso de un sistema o
graficas que te muestran aproximadamente cuales son los datos que apuebas asi como cuales datos no aprueban con su debida grafica de los indicacores del excel.

Ejemplo

- Grafica de tasa promedio de RATIO de DTI rechazada

- Grafica de tasa de aprobaciones de acuerdo al ingreso

- Media de puntaje crediticio aprobado

- Tasa de prestamos rechazados por monto solicitado fuera de los parametros


Indica si el modelo fue efectivo en la tarea propuesta y qué mejoras se pueden hacer.

### Desafios encontrados:
Menciona dificultades en el diseño, entrenamiento o filtrado de datos.

Filtrado de los datos

- Uso de los strings en campos como texto:

  Esto en especial en: ***Text,Employment_Status y Approval*** porque estos campos tenian valores que debian ser evaluados pero el modelo de la libreria de **keras**. Haciendo que diera errores. La mejor forma que encontramos para el procesamiento de los datos fue a traves de la siguiente linea de codigo:

  (Para mas detalles ver el paso 1)


```python
from sklearn.preprocessing import LabelEncoder
...
# De esta forma se hace una asignacion automatica de los valores de string
# fuente: la comunidad de stackoverflow
keywords=["start", "invest"]

print("🔎 Palabras clave más frecuentes:", keywords)

# Función para marcar si un texto contiene palabras clave
def contains_keywords(text, keywords):
    text = text.lower()
    return 1 if any(word in text for word in keywords) else 0

# Funcion para clasificar a las personas de empleados y no empleados
def is_employed(person):
  if(person=="employed"):
    return 1
  else:
    return 0

# Aplicar la función al DataFrame
X = X.copy() # Se coloca la copia para evitar que se lance el error al filtrar los datos
X["Text"] = dataframe["Text"].apply(lambda x: contains_keywords(x, keywords))

X["Employment_Status"] = dataframe["Employment_Status"].apply(lambda x: is_employed(x))

...  
```

> 📌 **Nota Importante:**  
> La librería requiere **valores numéricos obligatoriamente** para funcionar correctamente.  
> Sin este manejo, la transformación fallaría y el sistema no aceptaría los datos.
> Este fallo tambien nos paso haciendo la interfaz para los usuarios en dichos campos de nuevo ***Text,Employment_Status y Approval***



- Normalización de los datos: Debido a que al manejar datos como es el caso de ingresos que pueden pasar de 10.000 a 1.000 o incluso mas como 100.000$ termina siendo montos muy grandes que deben de ser tratados y manejados. La mejor forma que encontramos para el procesamiento de los datos fue a traves de la siguiente linea de codigo:

  (Para mas detalles ver el paso 3)

```python
from sklearn.model_selection import train_test_split #separador de casos para entrenamiento y prueba
from sklearn.preprocessing import StandardScaler
...

# Scale the data
scaler = StandardScaler()

# Se normalizan los datos a entrenar
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

...
```

> 📌 **Nota Importante:**  
> De esa forma nos ahorramos el problema de la gran diferencia entre los datos

### Propón mejoras futuras.

Sugerir posibles optimizaciones en el modelo y en la implementación del programa.

De acuerdo a la IA la clasificacion de la informacion deberia de hacercse usando una IA de analicis de lenguaje, de forma que el genere esas relaciones de las palabras claves.

Otra opcion como dejamos en la documentacion en el paso 2

```python
...
# Filtrar solo los textos aprobados
approved_texts = " ".join(dataframe[dataframe["Approval"] == "Approved"]["Text"]).lower()

# Limpiar texto (eliminar signos de puntuación)
approved_texts = re.sub(r'[^\w\s]', '', approved_texts)

# Separar palabras y contar frecuencia
word_counts = Counter(approved_texts.split())

# Otro criterio
# Obtener las 5 palabras más comunes de los textos aprobados En caso que se quiera hacer con ese criterio
# Si se quiere hacer para los mas comunes

# keywords = [word for word, count in word_counts.most_common(5)]
...
```


> 💡 **Idea:**  
> Hacer que los datos de las palabras mas encontradas en el documento guardarlo en otro csv o en un txt y de ahi tomarlo como los ***keywords*** para poder sacar los datos de forma dinamica y haga el criterio de acuerdo a los datos el propio modelo,

# Criterio de Evaluación

| 1-4: Insuficiente | 5-8: En Desarrollo | 9-12: Satisfactorio | 13-16: Bueno | 17-20: Sobresaliente |

| 1. Diseño y Entrenamiento del Modelo MLP (8 puntos) | El modelo no se implementa o presenta errores significativos. No se evidencia entrenamiento o el rendimiento es muy pobre. (1-2) | El modelo se implementa con errores y el entrenamiento es básico. El rendimiento es bajo y la comprensión de los hiperparámetros es limitada. (3-5) | Se implementa un modelo MLP funcional y se realiza un entrenamiento básico. El rendimiento es aceptable, y se intenta ajustar algunos hiperparámetros. (6-8) | Se diseña e implementa un modelo MLP robusto. Se realiza un entrenamiento adecuado y se exploran varios hiperparámetros para mejorar el rendimiento. (9-12) | Se diseña e implementa un modelo MLP altamente efectivo y optimizado. Se realiza un entrenamiento exhaustivo y se aplica un tuning de hiperparámetros sistemático utilizando Keras Tuner u otras técnicas avanzadas, logrando un rendimiento sobresaliente. (13-16) | Se demuestra un diseño excepcional del modelo MLP, con una justificación clara de la arquitectura y las funciones de activación. El entrenamiento es riguroso y la optimización de hiperparámetros es exhaustiva y bien documentada, resultando en un modelo con un rendimiento superior y generalización robusta. (17-20) |


| 2. Filtrado de Datos para Negocios (4 puntos) | No se realiza ningún filtrado de datos o el intento es incorrecto. (1) | Se intenta filtrar los datos, pero el método es ineficiente o incorrecto, incluyendo datos irrelevantes. (2) | Se realiza un filtrado básico de los datos, identificando algunas instancias relevantes para negocios, pero aún quedan datos no deseados. (3) | Se implementa una estrategia efectiva para filtrar los datos, identificando la mayoría de las instancias relevantes para negocios utilizando palabras clave como "invest" y "start". (4) | Se demuestra una comprensión profunda de la tarea de filtrado, implementando una estrategia sofisticada y precisa que aísla eficazmente los datos relevantes para préstamos de negocios, justificando las palabras clave y la lógica utilizada. (4) |


| 3. Programa de Interacción con el Usuario (4 puntos) | El programa no se implementa o falla al recibir entradas. No se manejan errores de entrada. (1) | Se implementa un programa básico que solicita entradas, pero no maneja entradas inválidas y puede arrojar errores. La recomendación es simple o inexistente. (2) | El programa solicita entradas clave y proporciona una recomendación básica. Se intenta manejar algunas entradas inválidas, pero no de forma exhaustiva. (3) | El programa guía al usuario para ingresar los datos clave de manera clara y amigable. Implementa una validación robusta para corregir entradas inválidas sin errores. La recomendación es clara y se basa en la salida del modelo. (4) | El programa ofrece una experiencia de usuario intuitiva y robusta. La validación de entradas es exhaustiva y proporciona retroalimentación útil al usuario. La recomendación es detallada y contextualizada, demostrando una integración perfecta con el modelo predictivo. (4) |


| 4. Calidad y Claridad del Código (4 puntos) | El código es ilegible, sin comentarios o con comentarios incorrectos. No se identifica el código generado por IA. (1) | El código es difícil de seguir, con pocos comentarios o comentarios vagos. La identificación del código generado por IA es inconsistente o ausente. (2) | El código es generalmente legible y contiene algunos comentarios útiles. Se intenta identificar el código generado por IA, pero la explicación puede ser superficial. (3) | Todo el código está debidamente comentado, explicando la lógica de cada sección y variable. Se identifica claramente el código generado por IA y se explica su funcionamiento línea por línea. (4) | El código demuestra una organización y claridad excepcionales. Los comentarios son detallados y perspicaces, facilitando la comprensión del flujo y la lógica del programa. La identificación y explicación del código generado por IA son exhaustivas y demuestran un entendimiento profundo del código. (4) |

Puntuación Total Posible: 20 puntos



##Tips del Profesor

NT: Hay posiblidad que el modelo quede muy pequeño y hay que determinar que hacer en caso que haya underfetting

Es un modelo de clasificacion

1. ¿Cuantas personas quieren invertir en el negocio? 500 es buen camino
2. ¿ Hay outlayers en las variables? ¿ En cuales variables? ¿Como las voy a calcular?
(la libreria del profesor tiene una funcion para saber cuales son los outlayers)

3. ¿Que hago con las variables cualitativas? (categorias) Ej creo hacer que categoria 1 = 1 y asi

**OJO: No puede haber un desbalanceo de los datos osea 2 datos 50/50 3 datos 33/33/33**

Librerias recomendadas: Keras, sidekid, kerastunner

**PISTA: Lo que hable de expandir, mejorar, etc son elmentos que probablemnte si se tengas que prestar**
